In [1]:
import iris
import iris.plot as iplt
import iris.quickplot as qplt
import matplotlib.pyplot as plt
import sys
sys.path.append('/home/nick/python/ASoP_global/ASoP-Coherence')
from asop_coherence_global_temporal import get_asop_dict
import matplotlib.cm as mpl_cm
import numpy as np
import numpy.ma as ma
import pandas as pd

In [2]:
def load_summary_metric(filename,constraint,new_long_name,new_units):
    cube = iris.load_cube(filename,constraint)
    cube.long_name = new_long_name
    cube.units = new_units
    return(cube)

In [3]:
def load_all_summary_metrics(asop_dict,wet_season_threshold='1d24'):
    constraints_longnames_units = [
        ('Temporal intermittency on-off metric based on 4 divisions (mean of all months in wet season)','Temporal coherence metric, 4 divs, all wet season, 3hr - '+asop_dict['name'],'1'),
        ('Probability of upper division followed by upper division (mean of all months in wet season)','p(on|on), 4 divs, all wet season, 3hr '+asop_dict['name'],'1'),
        ('Probability of upper division followed by lower division (mean of all months in wet season)','p(on|off), 4 divs, all wet season, 3hr '+asop_dict['name'],'1'),
        ('Probability of lower division followed by upper division (mean of all months in wet season)','p(off|on), 4 divs, all wet season, 3hr '+asop_dict['name'],'1'),
        ('Probability of lower division followed by lower division (mean of all months in wet season)','p(off|off), 4 divs, all wet season, 3hr '+asop_dict['name'],'1')
    ]
    out_cubelist = []
    summary_file = asop_dict['desc']+'_asop_temporal_summary_wetseason'+wet_season_threshold+'.nc'
    for constraint,long_name,units in constraints_longnames_units:
        cube = load_summary_metric(str(asop_dict['dir']/summary_file),constraint,long_name,units)
        out_cubelist.append(cube)
    return(out_cubelist)

In [4]:
def find_autocorr_threshold(cube,lag_length,threshold=0.5,long_name=None,units=None):
    import numpy as np
    #max_autocorr = cube.collapsed('lag',iris.analysis.MEAN).copy()
    lon = cube.coord('longitude')
    lat = cube.coord('latitude')
    nlon = len(lon.points)
    nlat = len(lat.points)
    nlag = len(cube.coord('lag').points)
#    max_autocorr = iris.cube.Cube(np.ma.empty((nlat,nlon)),dim_coords_and_dims=[(lat,0),(lon,1)])
    max_autocorr = np.zeros((nlat,nlon))
    for y in range(nlat):
        for x in range(nlon):
            below_threshold = np.where(cube.data[:,y,x] <= threshold)
            if np.sum(below_threshold) >= 1:
                max_autocorr[y,x] = np.amin(below_threshold)*lag_length+0.5
            else:
                max_autocorr[y,x] = nlag*lag_length+0.5
    max_autocorr_cube = iris.cube.Cube(data=max_autocorr,dim_coords_and_dims=[(lat,0),(lon,1)],long_name=long_name,units=units,var_name='autocorr_threshold'+str(threshold))
    return(max_autocorr_cube) 

In [5]:
def plot_summary_metric(model_cube,obs_cube,model_dict,obs_dict,raw_levs,diff_levs,raw_cmap,diff_cmap):
    fig = plt.figure()
    raw_cmap = mpl_cm.get_cmap(raw_cmap)
    diff_cmap = mpl_cm.get_cmap(diff_cmap)
    qplt.contourf(model_cube,raw_levs,cmap=raw_cmap)
    plt.gca().coastlines()
    plt.savefig('plots/asop_coherence_global_temporal_'+model_dict['name']+'_'+model_cube.var_name+'.png',dpi=200)

    fig = plt.figure()
    qplt.contourf(obs_cube,raw_levs,cmap=raw_cmap)
    plt.gca().coastlines()
    plt.savefig('plots/asop_coherence_global_temporal_'+obs_dict['name']+'_'+obs_cube.var_name+'.png',dpi=200)

    fig = plt.figure()
    diff_cube = model_cube-obs_cube
    diff_cube.long_name=model_cube.long_name+' diff '+obs_dict['name']
    qplt.contourf(diff_cube,diff_levs,cmap=diff_cmap)
    plt.gca().coastlines()
    plt.savefig('plots/asop_coherence_global_temporal_'+model_dict['name']+'-minus-'+obs_dict['name']+'_'+model_cube.var_name+'.png',dpi=200)

In [8]:
def compute_summary_stats(model_cube,obs_cube,region,region_type,region_name,diag_name,model_dict,mask=None):
    import pandas as pd
    # Average space-time summary metrics over a given region.
    # Mask for land-only if requested.
    import iris.analysis.stats as istats
    grid_constraint = iris.Constraint(latitude=lambda cell: region[0] <= cell <= region[1],
                                      longitude=lambda cell: region[2] <= cell <= region[3])
    model_region = model_cube.extract(grid_constraint)
    obs_region = obs_cube.extract(grid_constraint)
    if region_type == 'land' or region_type == 'ocean':
        if mask is None:
            raise Exception('Computing summary stats over '+region_type+' requires a mask, but mask is None.')
        mask_region = mask.extract(grid_constraint)
        mask_region.coord('longitude').guess_bounds()
        mask_region.coord('latitude').guess_bounds()
        mask_region = mask_region.regrid(model_region,iris.analysis.AreaWeighted())
        if region_type == 'land':
            model_region = model_region.copy(data=np.ma.array(model_region.data,mask=np.where(mask_region.data > 0.5,False,True)))
            obs_region = obs_region.copy(data=np.ma.array(obs_region.data,mask=np.where(mask_region.data > 0.5,False,True)))
        if region_type == 'ocean':
            model_region = model_region.copy(data=np.ma.array(model_region.data,mask=np.where(mask_region.data < 0.5,False,True)))
            obs_region = obs_region.copy(data=np.ma.array(obs_region.data,mask=np.where(mask_region.data < 0.5,False,True)))
    weights = iris.analysis.cartography.area_weights(model_region)
    model_avg = model_region.collapsed(['longitude','latitude'],iris.analysis.MEAN,weights=weights)
    obs_avg = obs_region.collapsed(['longitude','latitude'],iris.analysis.MEAN,weights=weights)
    bias = model_avg-obs_avg
    diff = model_region-obs_region
    rmse = diff.collapsed(('longitude','latitude'),iris.analysis.RMS,weights=weights)
    pcorr = istats.pearsonr(model_region,obs_region) #,corr_coords=('latitude','longitude'))
    metric_dict = pd.Series({
        'model_name': model_dict['name'],
        'diag_name': diag_name,
        'region_name': region_name,
        'bias': bias.data,
        'rmse': rmse.data,
        'pattern_corr': pcorr.data
    },name=region_name)
    return(metric_dict)

In [9]:
regions = [
    ([-30,30,0,360],'both','trop'),
    ([-60,-30,0,360],'both','sh'),
    ([30,60,0,360],'both','nh'),
    ([-60,60,0,360],'both','glob'),
    ([-30,30,0,360],'land','trop_land'),
    ([-30,30,0,360],'ocean','trop_ocean'),
    ([-60,-30,0,360],'land','sh_land'),
    ([-60,-30,0,360],'ocean','sh_ocean'),
    ([30,60,0,360],'land','nh_land'),
    ([30,60,0,360],'ocean','nh_ocean'),
    ([-60,60,0,360],'land','glob_land'),
    ([-60,60,0,360],'ocean','glob_ocean')
]
mask_file='/media/nick/lacie_tb3/datasets/land_sea_mask/landfrac_n216e_hadgem3-10.3.nc'
mask = iris.load_cube(mask_file,'land_area_fraction')
metrics = pd.DataFrame()

In [6]:
awi_dict = get_asop_dict('BCC')
gpm_dict = get_asop_dict('GPM_IMERG')
awi_temporal_summary,awi_ponon,awi_ponoff,awi_poffon,awi_poffoff = load_all_summary_metrics(awi_dict)
gpm_temporal_summary,gpm_ponon,gpm_ponoff,gpm_poffon,gpm_poffoff = load_all_summary_metrics(gpm_dict)

In [7]:
awi_autocorr_filename = str(awi_dict['dir'])+'/'+awi_dict['desc']+'_asop_temporal_autocorr_wetseason1d24.nc'
awi_autocorr = load_summary_metric(awi_autocorr_filename,'autocorr_wetseason_precip_mean',None,'Hours')
gpm_autocorr_filename = str(gpm_dict['dir'])+'/'+gpm_dict['desc']+'_asop_temporal_autocorr_wetseason1d24.nc'
gpm_autocorr = load_summary_metric(gpm_autocorr_filename,'autocorr_wetseason_precip_mean',None,'Hours')

In [10]:
autocorr_thresholds=[0.1,0.2,0.3,0.4,0.5]
for threshold in autocorr_thresholds:
    awi_autocorr_threshold = find_autocorr_threshold(awi_autocorr,3,threshold=threshold,long_name='Temporal autocorr metric, >'+str(threshold)+', all wet season, 3hr - AWI',units='Hours')
    gpm_autocorr_threshold = find_autocorr_threshold(gpm_autocorr,3,threshold=threshold,long_name='Temporal autocorr metric, >'+str(threshold)+', all wet season, 3hr - GPM',units='Hours')
    if threshold <= 0.3:
        autocorr_raw_levels=[3,6,9,12,15,18,21,24,27,30,33,36,39,42,45,48]
        autocorr_diff_levels=[-45,-35,-25,-20,-15,-11,-8,-5,-3,-1,1,3,5,8,11,15,20,25,35,45]
    else:
        autocorr_raw_levels=[3,6,9,12,15,18,21,24,27]
        autocorr_diff_levels=[-15,-13,-11,-9,-7,-5,-3,-1,1,3,5,7,9,11,13,15]
    plot_summary_metric(awi_autocorr_threshold,gpm_autocorr_threshold,awi_dict,gpm_dict,autocorr_raw_levels,autocorr_diff_levels,'Oranges','brewer_PRGn_11')
    for region,region_type,region_name in regions:
        metric_dict = compute_summary_stats(awi_autocorr_threshold,gpm_autocorr_threshold,region,region_type,region_name,'autocorr_threshold'+str(threshold),awi_dict,mask=mask)
        metrics = metrics.append(metric_dict)

In [11]:
prob_raw_levs=[0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9]
prob_diff_levs=[-0.45,-0.35,-0.25,-0.15,-0.05,0.05,0.15,0.25,0.35,0.45]
plot_summary_metric(awi_temporal_summary,gpm_temporal_summary,awi_dict,gpm_dict,prob_raw_levs,prob_diff_levs,'Oranges','brewer_PRGn_11')
plot_summary_metric(awi_ponon,gpm_ponon,awi_dict,gpm_dict,prob_raw_levs,prob_diff_levs,'Oranges','brewer_PRGn_11')
plot_summary_metric(awi_ponoff,gpm_ponoff,awi_dict,gpm_dict,prob_raw_levs,prob_diff_levs,'Oranges','brewer_PRGn_11')
plot_summary_metric(awi_poffon,gpm_poffon,awi_dict,gpm_dict,prob_raw_levs,prob_diff_levs,'Oranges','brewer_PRGn_11')
plot_summary_metric(awi_poffoff,gpm_poffoff,awi_dict,gpm_dict,prob_raw_levs,prob_diff_levs,'Oranges','brewer_PRGn_11')
#for region,region_type,region_name in regions:
#    metric_dict = compute_summary_stats(awi_temporal_summary,gpm_temporal_summary,region,region_type,region_name,'temporal_summary',awi_dict,mask=mask)
#    metrics = metrics.append(metric_dict)
#    metric_dict = compute_summary_stats(awi_ponon,gpm_ponon,region,region_type,region_name,'temporal_ponon',awi_dict,mask=mask)
#    metrics = metrics.append(metric_dict)
#    metric_dict = compute_summary_stats(awi_ponoff,gpm_ponoff,region,region_type,region_name,'temporal_ponoff',awi_dict,mask=mask)
#    metrics = metrics.append(metric_dict)
#    metric_dict = compute_summary_stats(awi_poffon,gpm_poffon,region,region_type,region_name,'temporal_poffon',awi_dict,mask=mask)
#    metrics = metrics.append(metric_dict)
#    metric_dict = compute_summary_stats(awi_poffoff,gpm_poffoff,region,region_type,region_name,'temporal_poffoff',awi_dict,mask=mask)
#    metrics = metrics.append(metric_dict)

In [12]:
test = metrics[(metrics['diag_name'] == 'temporal_summary')]
from tabulate import tabulate
print(tabulate(test[['bias','rmse','pattern_corr']]))

----------  -----------  ---------  ----------
trop        -0.0318547   0.063953   0.402621
sh           0.0621001   0.0671219  0.848728
nh           0.0771838   0.0870889  0.0362439
glob         0.0136853   0.0721513  0.133021
trop_land   -0.0101847   0.062169   0.406458
trop_ocean  -0.0382266   0.0644441  0.366701
sh_land      0.0772411   0.0874116  0.300395
sh_ocean     0.0613056   0.0663176  0.86067
nh_land      0.105455    0.111476   0.00846727
nh_ocean     0.0599371   0.0687184  0.098693
glob_land    0.0424168   0.0865429  0.169274
glob_ocean   0.00512745  0.0674006  0.134872
----------  -----------  ---------  ----------


In [13]:
test = metrics[(metrics['diag_name'] == 'autocorr_threshold0.5')]
print(tabulate(test[['bias','rmse','pattern_corr']]))

----------  ----------  --------  ---------
trop        -0.138935   12.9927   0.679886
sh           0.06422     8.29282  0.60124
nh           0.278608   11.1757   0.687322
glob         0.0250444  11.4203   0.678223
trop_land    1.68038    11.9188   0.786522
trop_ocean  -0.687042   13.2992   0.636211
sh_land     -9.02862    24.1457   0.0931557
sh_ocean     0.50623     6.61613  0.704374
nh_land     -0.957806   14.7869   0.6238
nh_ocean     1.14949     7.67194  0.7802
glob_land    0.149829   13.8676   0.684171
glob_ocean  -0.0174959  10.4559   0.659687
----------  ----------  --------  ---------


In [14]:
test = metrics[(metrics['diag_name'] == 'autocorr_threshold0.2')]
print(tabulate(test[['bias','rmse','pattern_corr']]))

----------  ---------  --------  --------
trop         2.05296   11.9643   0.648006
sh           0.801367   7.34762  0.653327
nh           1.71119    9.96728  0.691292
glob         1.72406   10.4562   0.684801
trop_land    4.04548   12.1237   0.751715
trop_ocean   1.45268   11.9159   0.604156
sh_land     -6.24719   20.6252   0.101673
sh_ocean     1.144      5.99442  0.741862
nh_land      1.68363   12.9862   0.637934
nh_ocean     1.7306     7.10905  0.76762
glob_land    2.6437    12.9641   0.672076
glob_ocean   1.41055    9.45035  0.687956
----------  ---------  --------  --------
